In [1]:
import pandas as pd
from sqlalchemy import create_engine
# import psycopg2

### Extract CSVs into DataFrames

In [2]:
eurovision_file = "data/eurovision_scores.csv"
countries_file = "data/countries.csv"
eurovision_df = pd.read_csv(eurovision_file)
countries_df = pd.read_csv(countries_file)
eurovision_df.head()

,Year,(semi-) final,Edition,Jury or Televoting,From country,To country,Points,Unnamed: 7,Unnamed: 8
0,1975,f,1975f,J,Belgium,Belgium,0,NaN,NaN
1,1975,f,1975f,J,Belgium,Finland,0,NaN,NaN
2,1975,f,1975f,J,Belgium,France,2,NaN,NaN
3,1975,f,1975f,J,Belgium,Germany,0,NaN,NaN
4,1975,f,1975f,J,Belgium,Ireland,12,NaN,NaN


### Transform premise DataFrame

In [3]:
eurovision_df['id'] = range(1, len(eurovision_df) + 1)

# p/remise_cols = ["id”, "Year, “(semi-) final”, “Edition”, “Jury or Televoting”, “From country”, “To country”, “Points” ]
premise_cols = ["id", "Year", "(semi-) final", "Edition", "Jury or Televoting", "From country", "To country", "Points" ]

eurovision_data= eurovision_df[premise_cols].copy()

# Rename the column headers
eurovision_data = eurovision_data.rename(columns={"(semi-) final": "rounds",
                                                          "Edition" : "edition",
                                                          "Jury or Televoting": "jury_or_televoting",
                                                          "From country": "from_country",
                                                          "To country" : "to_country",
                                                          "Points" : "points",
                                                          "Year" : "year"})
# eurovision_data.applymap(str)
eurovision_data.head()

,id,year,rounds,edition,jury_or_televoting,from_country,to_country,points
0,1,1975,f,1975f,J,Belgium,Belgium,0
1,2,1975,f,1975f,J,Belgium,Finland,0
2,3,1975,f,1975f,J,Belgium,France,2
3,4,1975,f,1975f,J,Belgium,Germany,0
4,5,1975,f,1975f,J,Belgium,Ireland,12


In [4]:
worldmap = ["id", "year", "to_country" , "points"]
transformed_df = eurovision_data[worldmap].copy()
transformed_df["year"]=transformed_df["year"].astype(str)

transformed_df = transformed_df.groupby(['year','to_country']).agg({'points' : 'sum'})
worldmap_df = transformed_df.reset_index()
worldmap_df.head()
# list(worldmap_df.columns)

,year,to_country,points
0,1975,Belgium,17
1,1975,Finland,74
2,1975,France,91
3,1975,Germany,15
4,1975,Ireland,68


In [5]:
countries_df.head()
worldwide_df = pd.merge(worldmap_df, countries_df[["latitude", "longitude", "name"]], left_on = "to_country", right_on = "name", how ="left")

worldwide_df = worldwide_df.dropna(how = "any")
worldwide_df = worldwide_df.drop(["name"], axis=1)
worldwide_df

,year,to_country,points,latitude,longitude
0,1975,Belgium,17,50.503887,4.469936
1,1975,Finland,74,61.924110,25.748151
2,1975,France,91,46.227638,2.213749
3,1975,Germany,15,51.165691,10.451526
4,1975,Ireland,68,53.412910,-8.243890
...,...,...,...,...,...
1129,2016,Spain,77,40.463667,-3.749220
1130,2016,Sweden,261,60.128161,18.643501
1131,2016,Switzerland,28,46.818188,8.227512
1133,2016,Ukraine,821,48.379433,31.165580


### Create database connection

In [6]:
# postgres_string = "postgres:password@localhost:5432/eurovision_db"
# postgres_engine = create_engine(f'postgresql://{postgres_string}')

In [7]:
connection_string = "db/eurovision.sqlite"
engine = create_engine(f'sqlite:///{connection_string}')

### Load DataFrames into database

In [8]:
# eurovision_data.to_sql(name='eurovision', con=engine, if_exists='append', index=True)

In [10]:
engine.execute('''CREATE TABLE eurovision (
       id INT,
       year INT,
       rounds TEXT,
       edition TEXT,
       jury_or_televoting TEXT,
       from_country TEXT,
       to_country TEXT,
       points INT,
       primary key("id"));''')

eurovision_data.to_sql(name='eurovision', con=engine, index=False, if_exists='append')

eurovision_data.head()

,id,year,rounds,edition,jury_or_televoting,from_country,to_country,points
0,1,1975,f,1975f,J,Belgium,Belgium,0
1,2,1975,f,1975f,J,Belgium,Finland,0
2,3,1975,f,1975f,J,Belgium,France,2
3,4,1975,f,1975f,J,Belgium,Germany,0
4,5,1975,f,1975f,J,Belgium,Ireland,12


In [11]:
engine.execute('''CREATE TABLE worldmapdata (
       id INT,
       year INT,
       to_country TEXT,
       points INT,
       latitude FLOAT,
       longitude FLOAT,
       primary key("id"));''')

worldwide_df.to_sql(name='worldmapdata', con=engine, index=False, if_exists='append')
worldwide_df.head()


,year,to_country,points,latitude,longitude
0,1975,Belgium,17,50.503887,4.469936
1,1975,Finland,74,61.924110,25.748151
2,1975,France,91,46.227638,2.213749
3,1975,Germany,15,51.165691,10.451526
4,1975,Ireland,68,53.412910,-8.243890
